Nama     : **Muhamad Ilyas**               
Email    : **181240000831@unisnu.ac.id**   
Domisili : **Kabupaten Jepara, Jawa Tengah**

In [1]:
#memanggil file dengan drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#membuat dataframe
import pandas as pd

path = '/content/drive/MyDrive/Dataset/bbc-news-data.csv'
df = pd.read_csv(path, sep='\t')
df

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...
...,...,...,...,...
2220,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...
2221,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...
2222,tech,399.txt,Be careful how you code,A new European directive could put software w...
2223,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...


In [3]:
#menghapus kolom yang tidak diperlukan
df.drop(['filename', 'content'], axis=1, inplace=True)

In [4]:
#melihat info file
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   title     2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [5]:
#melihat file
df.head()

,category,title
0,business,Ad sales boost Time Warner profit
1,business,Dollar gains on Greenspan speech
2,business,Yukos unit buyer faces loan claim
3,business,High fuel prices hit BA's profits
4,business,Pernod takeover talk lifts Domecq


In [6]:
#melakukan one-hot-encoding dan membuat dataframe baru
category = pd.get_dummies(df.category)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='category')
df_baru

,title,business,entertainment,politics,sport,tech
0,Ad sales boost Time Warner profit,1,0,0,0,0
1,Dollar gains on Greenspan speech,1,0,0,0,0
2,Yukos unit buyer faces loan claim,1,0,0,0,0
3,High fuel prices hit BA's profits,1,0,0,0,0
4,Pernod takeover talk lifts Domecq,1,0,0,0,0
...,...,...,...,...,...,...
2220,BT program to beat dialler scams,0,0,0,0,1
2221,Spam e-mails tempt net shoppers,0,0,0,0,1
2222,Be careful how you code,0,0,0,0,1
2223,US cyber security chief resigns,0,0,0,0,1


In [8]:
df_baru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          2225 non-null   object
 1   business       2225 non-null   uint8 
 2   entertainment  2225 non-null   uint8 
 3   politics       2225 non-null   uint8 
 4   sport          2225 non-null   uint8 
 5   tech           2225 non-null   uint8 
dtypes: object(1), uint8(5)
memory usage: 28.4+ KB


In [9]:
#mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array
berita = df_baru['title'].values
label = df_baru[['business', 'entertainment', 'politics', 'sport', 'tech']].values

In [10]:
label

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]], dtype=uint8)

In [14]:
#data training dan data test
from sklearn.model_selection import train_test_split
berita_latih, berita_test, label_latih, label_test = train_test_split(berita, label, test_size=0.2)

In [51]:
#mengubah dataset menjadi bilangan numerik
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 

tokenizer = Tokenizer(num_words=10000, oov_token='x')
tokenizer.fit_on_texts(berita_latih)
tokenizer.fit_on_texts(berita_test)

sekuens_latih = tokenizer.texts_to_sequences(berita_latih)
sekuens_test = tokenizer.texts_to_sequences(berita_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

In [52]:
#arsitektur sequential
import tensorflow as tf
from tensorflow.keras.layers import Dropout
from keras import regularizers
model = tf.keras.Sequential([
      tf.keras.layers.Embedding(input_dim=10000, output_dim=32),
      tf.keras.layers.LSTM(64),
      tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(5, activation='softmax')
])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_12 (Dense)             (None, 512)               33280     
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 5)                

In [53]:
#menentukan optimizer dan loss function dari model
from tensorflow import keras
optim = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optim,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [54]:
#memanggil Callback
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.95):
      print("\nTraining sudah mencapai akurasi diatas 95%")
      self.model.stop_training = True
callbacks = myCallback()

In [63]:
#melatih model dengan fungsi fit
num_epochs=30
history = model.fit(padded_latih, label_latih, epochs=num_epochs,
                    validation_data=(padded_test, label_test),
                    verbose=2,
                    callbacks=[callbacks],
                    batch_size=256)

Epoch 1/30
7/7 - 0s - loss: 0.5695 - accuracy: 0.9888 - val_loss: 1.7136 - val_accuracy: 0.7596

Training sudah mencapai akurasi diatas 95%
